<!--
tags: [tutorial]
description: Code Executors
-->

# Code Executors

**This chapter is a work in progress, will be updated once the containerized code executor is ready]**

In the [last chapter](./human-in-the-loop), 
we used two agents powered by a large language model (LLM) 
to play a game. It was a fun exercise, but LLM by itself can only
generate text -- it cannot interact with its environment or reliably perform
logical operations like arithmetic. 

In this chapter, we introduce code executor, an AutoGen's agent component that 
takes input messages with code blocks, performs execution, and output messages
with the results.

**Note**: Different from many other agent frameworks, AutoGen's code executor
can power an entire role of an agent: it can take input messages, execute code,
and output messages -- just like LLM.

In [ ]:
import os
import datetime
from IPython.display import Image
from autogen import ConversableAgent

Here is an example of a code executor agent:

In [ ]:
code_executor_agent = ConversableAgent(
    "code_executor_agent",
    llm_config=False,  # Turn off LLM for this agent.
    code_execution_config={"executor": "commandline-local"},
    human_input_mode="ALWAYS",  # Always take human input for this agent for safety.
)

The code executor agent is initialized with the `commandline-local` code executor.
Just like an agent powered by LLM, the code executor agent can also respond to 
messages.

In [ ]:
reply = code_executor_agent.generate_reply(
    messages=[
        {
            "role": "user",
            "content": """
```python
import random
print(random.randint(1, 100))
```
""",
        }
    ]
)
print(reply)

When you run the previous code block, you will be prompted for a human feedback,
which gives you an option to intercept message because the code executor can run
arbitrary code. It is important to have a human in the loop.

[Discuss containerization once that piece is ready.]

## User Proxy Agent and Assistant Agent

[Introduce the two-agent pattern of user proxy and assistant agents, and how does it connect to the code executor.]

## Agent and Coding

Writing and executing code is necessary for many complex tasks such as 
data analysis, machine learning, and mathematical modeling. 
In AutoGen, coding is a conversation between a code writer agent and a 
code executor agent, mirroring the interaction between a programmer and a
code interpreter.

The code writer agent can be powered by an LLM such as GPT-4 with code-writing
capability.
The following is an agent with a code writer role specified 
using `system_message`.

In [ ]:
code_writer_agent = ConversableAgent(
    "code_writer",
    system_message="You are an AI assistant. You write code in markdown code "
    "blocks. You do not execute code yourself, but always ask me to execute code "
    "for you in an IPython kernel.",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
    code_execution_config=False,  # Turn off code execution for this agent.
)

Here is an example of solving a math problem through a conversation
between the code writer and the code executor:

In [ ]:
result = code_executor_agent.initiate_chat(
    code_writer_agent,
    message="Write Python code to calculate the 14th Fibonacci number.",
)

During the previous chat session, human input was requested each time
the code executor agent responded to ensure that the code was safe to execute.

Now we can try a more complex example that involves querying the web.
Let's say we want to get the the stock prices year-to-date for
Tesla and Meta (formerly Facebook). We can also use the two agents
with several iterations of conversation.


In [ ]:
today = datetime.datetime.now().strftime("%Y-%m-%d")
result = code_executor_agent.initiate_chat(
    code_writer_agent, message=f"Today is {today}. Plot a chart for Tesla's and Microsoft's stock prices YTD."
)

Let's take a look at the chart that was generated.

In [ ]:
Image("31c88163ca9f414ca4ccb20045e4aa84.png")

## Containerization

Code executor is a powerful tool, but it can also be dangerous because it can affect the environment it runs it. For example, it can delete files, send data to a remote server, or even mine cryptocurrency. To mitigate these risks, we can run the code executor in a container. This way, the code executor can only access the resources that are explicitly given to it.

## Jupyter Executor

[Introduce the other built-in executor 'jupyter-executor']

## Summary

[Talk about tool executor and reference a separate chapter.]